# Ensemble Groundwater Predictions (EGP)
*R.A. Collenteur, Eawag, 2025*

This notebook shows how to use Pastas and meteorological ensemble forecasts to generate ensemble groundwater predictions (EGPs). The goal is to forecast the groundwater levels for a well in Switzerland (Gossau), one month ahead. Meteorological ensemble forecasts of the ECMWF with 51 ensemble members are used as data input. These members represent the uncertainty in the meteorological input data. The Pastas model is calibrated on 10 years of head data prior to the start of the forecast, using meteorological data provided by MeteoSwiss. In this example, meteorological forecasts are used, but it is straightforward to extend this to other input data such as ensembles of pumping forecasts.

<div class="alert alert-info">
<b>Note:</b>
Collenteur et al. (In Preparation) Ensemble groundwater predictions (EGP) in alluvial aquifers in Switzerland.
</div>

## 0. Import Python Packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import pastas as ps
import properscoring as psc

ps.set_log_level("ERROR")
ps.show_versions()

## 1. Load data

Below the head and meteorological data is loaded for the well in Gossau, Switzerland.



In [ ]:
head = pd.read_csv("data_forecast/heads.csv", index_col=0, parse_dates=True).squeeze()
prec = pd.read_csv("data_forecast/prec.csv", index_col=0, parse_dates=True).squeeze()
evap = pd.read_csv("data_forecast/evap.csv", index_col=0, parse_dates=True).squeeze()
temp = pd.read_csv("data_forecast/temp.csv", index_col=0, parse_dates=True).squeeze()

ps.plots.series(
    head,
    [prec, evap, temp],
    tmin="2004",
    tmax="2014",
    titles=False,
    labels=["Head\n[m]", "Prec.\n[mm/d]", "Evap.\n[mm/d]", "Temp.\n[C]"],
    table=True,
)
plt.tight_layout()

## Make Pastas Model

We now make a Pastas model to simulate the heads for this monitoring well in Gossau. Only meterological data, which is also available as forecast data, is used to model the groundwater levels. A nonlinear recharge model including a snow module is applied to compute the recharge. The model is calibrated on weekly groundwater level data in the period 2004-2014.

In [ ]:
ml = ps.Model(head)
ml.add_stressmodel(
    ps.RechargeModel(
        prec,
        evap,
        rfunc=ps.Gamma(),
        recharge=ps.rch.FlexModel(snow=True),
        temp=temp,
        name="rch",
    )
)

ml.set_parameter("rch_tt", vary=False)
ml.solve(tmin="2004", tmax="2014-02-01", report=True, fit_constant=False, freq_obs="1D")
ml.add_noisemodel(ps.ArNoiseModel())
ml.set_parameter("rch_srmax", vary=False)
ml.solve(
    tmin="2004", tmax="2014-02-01", initial=False, fit_constant=False, freq_obs="1D"
)

### Visualize the model results

In [ ]:
ml.plots.results();

## Prepare the forecast ensembles

Now that we have a calibrated Pastas model, we can prepare the forecast data used to generate the groundwater ensemble predictions. The forecast data should be prepared carefully as a dictionary. For each stressmodel, one item in the dictionary should be provided where the `key` is the stressmodel name (i.e., same as in `ml.stressmodels`) and the `value` a list of `pandas.DataFrames`. 

Each `DataFrame` should have the same `DateimeIndex` with the dates of the forecasts of the input data. It should also have the same number of columns, where each column represents an ensemble member (i.e., the precipitation and evaporation data need the same number of members). All these properties are internally checked and an error is raised if something is wrong.

In [ ]:
fc = {
    "rch": [
        pd.read_csv("data_forecast/ensemble_prec.csv", index_col=0, parse_dates=True),
        pd.read_csv("data_forecast/ensemble_evap.csv", index_col=0, parse_dates=True),
        pd.read_csv("data_forecast/ensemble_temp.csv", index_col=0, parse_dates=True),
    ]
}

fig, axes = plt.subplots(1, 3, figsize=(10, 3))
names = ["Cum. Precipitation", "Cum. Evaporation", "Temperature"]
for i in range(3):
    series = fc["rch"][i]
    if i < 2:
        series.cumsum().plot(legend=False, ax=axes[i], color="k", alpha=0.7)
    else:
        series.plot(legend=False, ax=axes[i], color="k", alpha=0.7)

    axes[i].set_title(names[i])

## Compute GWL forecasts

We can now generate the forecasts of the groundwater levels using the calibrated model. We can include the parameter uncertainty, by drawing multiple parameter sets (i.e., $N$ parameter sets). The model is run with each of the $N$ parameter sets for all $M$ ensemble member sets (i.e., set of precipitation, evaporation and temperature), resulting in $N$ x $M$ forecasts. 

For each individual forecast the mean forecast and the variance of the error or noise is returned, depending on whether or not `post_process` is `True` or `False`. If `True`, the forecast is 1) corrected using the last GWL measurement and the noisemodel, and 2) the variance is computed using the noise and the noisemodel. If `False`, the forecast is not corrected and the variance of the residuals is returned.



In [ ]:
# Draw parameter sets
params = ml.solver.get_parameter_sample(n=10)

# Generate the forecast
df_nopp = ps.forecast.forecast(ml, fc, params=params, post_process=False)
df_pp = ps.forecast.forecast(ml, fc, params=params, post_process=True)

The returned variable `df` is a DataFrame containing the ensemble groundwater predictions. The columns of `df` are a `MultiIndex` with the first row the $M$ ensemble members (i.e., a single meteorological forecast), the second row the $N$ parameter sets, and the third row three columns with the mean prediction and the lower and upper boundary of the prediction interval. 

## Compute the overall mean and variance

The forecast method returns a matrix with with $N$ x $M$ forecasts of the mean and variance. From these, we can compute the mean forecast easily. For the variance, which we can use to compute the (for example) the 95\% prediction interval, we apply the law of total variance. The forecast method provides a convenience method to do this: `ps.forecast.get_overall_mean_and_variance`. The method takes the output from the `ps.forecast.forecast` method and returns two `pandas.Series` with the overall mean and the variance.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 3), sharey=True)

for i, (df, label) in enumerate(zip([df_nopp, df_pp], ["No PP", "PP"])):
    mean, var = ps.forecast.get_overall_mean_and_variance(df)
    std = np.sqrt(var)

    ax = axes[i]
    ml.oseries.series.loc[df.index].plot(
        ax=ax, marker=".", color="k", linestyle="None", zorder=100
    )

    mean.plot(ax=ax, alpha=0.5, zorder=100)
    ax.plot(mean, color="k", label="Mean")
    ax.fill_between(
        mean.index,
        mean - std * 1.96,
        mean + std * 1.96,
        color="k",
        alpha=0.2,
        label="1 std",
    )

    df.loc[:, (slice(None), slice(None), "mean")].plot(
        color="gray", legend=False, ax=ax
    )

plt.legend(
    ["Observations", "mean forecast", "Ensemble Members", "95% Prediction interval"],
    loc="upper left",
)

plt.xlabel("Time")
plt.ylabel("Head [m]");